# Finding, and Dot Notation

In [6]:
from pymongo import MongoClient

client = MongoClient()
db = client.nobel
print(client)
print(db)

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)
Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel')


<br>

## Query operators

Query operators provide us more options when constructing filters. For example, we may wish to find documents where a field's value matches any of a set of options. To do this we can use the `$in` query operator.

- How many laureates were born in any of "Canada", "Mexico", or "USA"? 

In [7]:
#Save a filter for laureates who were born in with Canada, Mexico, or the USA.
criteria = {"bornCountry": {"$in": ["Canada", "Mexico", "USA"]}}

# Count them and save the count.
count = db.laureates.count_documents(criteria)
print(count)

295


If we wish to accept all but one option as a value for a field, we can use the `$ne` (**n**ot **e**qual) operator.

- How many laureates died in the USA but were not born in the USA?

In [9]:
# Save a filter for laureates who died in the USA and were not born there.
criteria = {"bornCountry": {"$ne": "USA"}, "diedCountry": "USA"}

# Count them and save the count.
count = db.laureates.count_documents(criteria)
print(count)

71


Below is a list of *comparison* query operators that are available in MongoDB:

`$eq` -- Matches values that are equal to a specified value. <br>
`$gt` -- Matches values that are greater than a specified value. <br>
`$gte` -- Matches values that are greater than or equal to a specified value. <br>
`$in` -- Matches any of the values specified in an array. <br>
`$lt` -- Matches values that are less than a specified value. <br>
`$lte` -- Matches values that are less than or equal to a specified value. <br>
`$ne` -- Matches all values that are not equal to a specified value. <br>
`$nin` -- Matches none of the values specified in an array.

<br>

In addition to comparison query operators, we can use *logical* query operators in our filters as well.

- How many laureates died in Canada ***OR*** were born in France?

In [10]:
#Save a filter for laureates who were born in Canada OR died in France.
criteria = {"$or":[{"bornCountry": "Canada"}, {"diedCountry": "France"}]}

# Count them and save the count.
count = db.laureates.count_documents(criteria)
print(count)

71


Below is a list of *logical* query operators that are available in MongoDB:

`$and` --	Joins query clauses with a logical AND returns all documents that match the conditions of both clauses. <br>
`$not` --	Inverts the effect of a query expression and returns documents that do not match the query expression. <br>
`$nor` --	Joins query clauses with a logical NOR returns all documents that fail to match both clauses. <br>
`$or` --	Joins query clauses with a logical OR returns all documents that match the conditions of either clause. 

See https://docs.mongodb.com/manual/reference/operator/query/ for more operators!

<br>

## Using dot notation

You will notice that the `prizes` field of a `laureates` document consists of an array of subdocuments:

An example `laureates` document:

```
{'_id': ObjectId('5b9ac94ff35b63cf5231ccb1'),
 'born': '1845-03-27',
 'bornCity': 'Lennep (now Remscheid)',
 'bornCountry': 'Prussia (now Germany)',
 'bornCountryCode': 'DE',
 'died': '1923-02-10',
 'diedCity': 'Munich',
 'diedCountry': 'Germany',
 'diedCountryCode': 'DE',
 'firstname': 'Wilhelm Conrad',
 'gender': 'male',
 'id': '1',
 'prizes': [{'affiliations': [{'city': 'Munich',
                               'country': 'Germany',
                               'name': 'Munich University'}],
             'category': 'physics',
             'motivation': '"in recognition of the extraordinary services '
                           'he has rendered by the discovery of the '
                           'remarkable rays subsequently named after him"',
             'share': '1',
             'year': '1901'}],
 'surname': 'Röntgen'}
```

An easy way to construct filters for array entries or subdocument fields is to use dot notation.

- Use the `$exists` operator to find out how many laureates were awarded at least two prizes? 

In [11]:
#Save a filter for laureates who were awarded at least two prizes. 
criteria = {"prizes.1":{"$exists": True}}

# Count them and save the count.
count = db.laureates.count_documents(criteria)
print(count)

6


- Estimate how many laureates were awarded at least two prizes in physics specifically? 

In [12]:
#Save a filter to estimate the number of laureates who were awarded at least two prizes in physics. 
criteria = {"prizes.0.category": "physics", "prizes.1.category": "physics"}

# Count them and save the count.
count = db.laureates.count_documents(criteria)
print(count)

1


In [13]:
db.laureates.find_one(criteria)

{'_id': ObjectId('5f1746222466f220f494d2d2'),
 'id': '66',
 'firstname': 'John',
 'surname': 'Bardeen',
 'born': '1908-05-23',
 'died': '1991-01-30',
 'bornCountry': 'USA',
 'bornCountryCode': 'US',
 'bornCity': 'Madison, WI',
 'diedCountry': 'USA',
 'diedCountryCode': 'US',
 'diedCity': 'Boston, MA',
 'gender': 'male',
 'prizes': [{'year': '1956',
   'category': 'physics',
   'share': '3',
   'motivation': '"for their researches on semiconductors and their discovery of the transistor effect"',
   'affiliations': [{'name': 'University of Illinois',
     'city': 'Urbana, IL',
     'country': 'USA'}]},
  {'year': '1972',
   'category': 'physics',
   'share': '3',
   'motivation': '"for their jointly developed theory of superconductivity, usually called the BCS-theory"',
   'affiliations': [{'name': 'University of Illinois',
     'city': 'Urbana, IL',
     'country': 'USA'}]}]}